# Task 1: POS

In [1]:
import nltk as nltk
from nltk import word_tokenize as wordtk
nltk.download('punkt')
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words =stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shubhangisrivastava/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shubhangisrivastava/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
uic= open('/Users/shubhangisrivastava/Documents/UIC Study material/IDS 566/Asg3/UIC.txt','r').read()
nyt= open('/Users/shubhangisrivastava/Documents/UIC Study material/IDS 566/Asg3/NYT.txt','r').read()
espn= open('/Users/shubhangisrivastava/Documents/UIC Study material/IDS 566/Asg3/ESPN.txt','r').read()

In [4]:
## things to remove 
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
tok = WordPunctTokenizer()
def text_cleaner(text):
    soup = BeautifulSoup(text, 'html.parser')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
   # for w in words:
     # if w not in stop_words:
      # return (" ".join(words)).strip()
    return (" ".join(words)).strip()


In [5]:
uic= (text_cleaner(uic)).split()
nyt= (text_cleaner(nyt)).split()
espn= (text_cleaner(espn)).split()

In [6]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shubhangisrivastava/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
#uic=[w for w in uic if not w in stop_words]
#nyt= [w for w in nyt if not w in stop_words] 
#espn=[w for w in espn if not w in stop_words] 

#### POS tags

In [8]:
tag_uic= nltk.pos_tag(uic)
tag_nyt= nltk.pos_tag(nyt)
tag_espn=nltk.pos_tag(espn)

print(len(tag_uic))
print(type(tag_uic))

5176
<class 'list'>


In [9]:
def get_freq(temp):
    tags = {}
    pos_list = list()
    for i in temp:
        pos_list.append(i[1])
    
    for i in pos_list:
        if i in tags.keys():
            tags[i] = tags[i] + 1
        else:
            tags[i] = 1
    return tags

    
tag_uic_freq = get_freq(tag_uic)
tag_nyt_freq = get_freq(tag_nyt)
tag_espn_freq = get_freq(tag_espn)

#print(tag_espn_freq)

In [10]:
#tag_uic_freq= nltk.FreqDist(tag for (word, tag) in tag_uic)
#tag_nyt_freq= nltk.FreqDist(tag for (word, tag) in tag_nyt)
#tag_espn_freq= nltk.FreqDist(tag for (word, tag) in tag_espn)


#print(tag_uic_freq)

### Jaccard Similarity 

In [15]:
def jaccard_similarity(d1,d2):
    num=list()
    den=list()
    for key in d1:
        for key1 in d2:
            intersection=0
            if(key==key1):
                intersection=min(d1[key],d2[key1])
                num.append(intersection)
                den.append(d1[key]+d2[key1]-intersection)
    return(np.sum(num)/np.sum(den))
#def jaccard_similarity(list1, list2):
 #   intersection = len(set(list1).intersection(list2))
  #  union = len(set(list1)) + len(set(list2)) - intersection

   # return intersection / union

#jaccard_similarity(tag_uic_freq , tag_nyt_freq)
#jaccard_similarity(tag_uic_freq , tag_espn_freq)
jaccard_similarity(tag_nyt_freq , tag_espn_freq)

0.33311389071757735

In [12]:
#jaccard_similarity(list( tag for (word, tag) in tag_uic), list(tag for (word,tag) in tag_nyt))

TypeError: list indices must be integers or slices, not str

In [ ]:
#jaccard_similarity(list( tag for (word, tag) in tag_uic), list(tag for (word,tag) in tag_espn))

# Task 2: Word Embedding and Visualization

Download Glove and prepare the file

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

Generation of tokens

In [ ]:
def genTokens(lines):
    tokens = []
    for line in lines:
        if len(line) > 0:
            tokens.append(wordtk(line.lower()))
    final = [item for sublist in tokens for item in sublist]
    return final

Tokens_UIC = genTokens(uic)

Getting the Glove Dictionary

In [ ]:
embeddings_dict = {}

In [ ]:
with open("/Users/shubhangisrivastava/Documents/UIC Study material/IDS 566/Asg3/glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

Randomly pick 50 words from the list of tokens and get their vectors


In [ ]:
from random import sample
random_tokens = sample(Tokens_UIC, 50)

uic_embeddings = {}
for key in random_tokens:
  try:
    uic_embeddings[key] = embeddings_dict[key]
  except:
    pass

Convert the dictionary to dataframe and perform PCA

In [ ]:
embed_df = pd.DataFrame(uic_embeddings.items(), columns=["word", "vector"])
print(embed_df)

def generateMatrix(embeddings_vector):
  matrix_rep = np.zeros((len(embeddings_vector), 50))
  for i in range(0, len(embeddings_vector)):
    for j in range(0,50):
      matrix_rep[i][j] = embeddings_vector[i][j]
    
  return matrix_rep

matrix_rep = generateMatrix(embed_df["vector"])

2-dimensional PCA Model

In [ ]:
from sklearn.decomposition import PCA
pca_four = PCA(n_components=2)
fit = pca_four.fit_transform(matrix_rep)
print(fit)

Data Visualization using matplotlib

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize= (12,12))
plt.plot([], marker="s")
plt.scatter(fit[:, 0], fit[:, 1])
for i, word in enumerate(uic_embeddings):
	plt.annotate(word, xy=(fit[i, 0], fit[i, 1]))
plt.show()